<a href="https://colab.research.google.com/github/arkeyan/Indicators/blob/main/Kompozit_Meta_Sinyal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install git+https://github.com/rongardF/tvdatafeed  backtesting
!pip install tradingview-screener==2.5.0

import numpy as np
import pandas as pd
from tvDatafeed import TvDatafeed, Interval
from tradingview_screener import get_all_symbols
import warnings

warnings.simplefilter(action='ignore')

# Zaman dilimi seçimi
def select_interval():
    interval_map = {
        "1": Interval.in_daily,
        "2": Interval.in_weekly,
        "3": Interval.in_monthly
    }
    print("\nLütfen zaman dilimini (interval) seçin:")
    print("1: Günlük")
    print("2: Haftalık")
    print("3: Aylık")
    choice = input("Seçiminiz (1/2/3): ")
    return interval_map.get(choice, Interval.in_daily)

# EMA hesaplama fonksiyonu
def ema(src, span):
    return src.ewm(span=span, adjust=False).mean()

# MACD hesaplama fonksiyonu
def macd(src, fast=12, slow=26, signal=9):
    macd_line = src.ewm(span=fast, adjust=False).mean() - src.ewm(span=slow, adjust=False).mean()
    signal_line = macd_line.ewm(span=signal, adjust=False).mean()
    return macd_line, signal_line

# MFI hesaplama fonksiyonu
def mfi(data, period=14):
    typical_price = (data['high'] + data['low'] + data['close']) / 3
    money_flow = typical_price * data['volume']
    positive_flow = money_flow.where(typical_price > typical_price.shift(1), 0)
    negative_flow = money_flow.where(typical_price < typical_price.shift(1), 0)

    positive_mf = positive_flow.rolling(window=period).sum()
    negative_mf = negative_flow.rolling(window=period).sum()
    mfi = 100 - (100 / (1 + positive_mf / negative_mf))
    return mfi

# TradingView bağlantısı ve hisse listesi
tv = TvDatafeed()
Hisseler = get_all_symbols(market='turkey')
Hisseler = [symbol.replace('BIST:', '') for symbol in Hisseler]
Hisseler = sorted(Hisseler)

# Kullanıcıdan zaman dilimi seçimi al
selected_interval = select_interval()

# Sonuçları saklamak için DataFrame
Titles = ['Hisse Adı', 'Son Fiyat', 'EMA Kontrol', 'RSI Al', 'MACD Al', 'MFI 50 Üzeri']
df_signals = pd.DataFrame(columns=Titles)

for hisse in Hisseler:
    try:
        # Seçilen zaman dilimine göre hisse ve XU100 verilerini alma
        data = tv.get_hist(symbol=hisse, exchange='BIST', interval=selected_interval, n_bars=100)
        dataref = tv.get_hist(symbol='XU100', exchange='BIST', interval=selected_interval, n_bars=100)
        data = data.reset_index()
        dataref = dataref.reset_index()

        # Fiyatları XU100 endeksine göre normalize etme
        result = data.copy()
        columns_to_divide = ['open', 'high', 'low', 'close', 'volume']
        for column in columns_to_divide:
            result[column] = 100 * (data[column] / dataref[column])

        # EMA değerlerini hesaplama
        result['EMA_34'] = ema(result['close'], 34)
        result['EMA_55'] = ema(result['close'], 55)
        result['EMA_144'] = ema(result['close'], 144)
        result['EMA_233'] = ema(result['close'], 233)

        # RSI ve MACD hesaplama
        result['RSI'] = result['close'].rolling(window=14).apply(lambda x: 100 - (100 / (1 + (x.diff().clip(lower=0).sum() / x.diff().clip(upper=0).abs().sum()))))
        result['MACD'], result['MACD_signal'] = macd(result['close'])
        result['MFI'] = mfi(result, 14)

        # Şartları kontrol etme
        latest_data = result.iloc[-1]
        ema_check = all(latest_data['close'] > latest_data[f'EMA_{ema}'] for ema in [34, 55, 144, 233])
        rsi_buy = latest_data['RSI'] > 50
        macd_buy = latest_data['MACD'] > latest_data['MACD_signal']
        mfi_check = latest_data['MFI'] > 50 and result['MFI'].iloc[-2] <= 50  # MFI'nin 50'yi yukarı kesmesi

        # Eğer tüm koşullar sağlanıyorsa sinyali kaydetme
        if ema_check and rsi_buy and macd_buy and mfi_check:
            L1 = [hisse, round(latest_data['close'], 2), ema_check, rsi_buy, macd_buy, mfi_check]
            df_signals.loc[len(df_signals)] = L1
            print(L1)  # Sadece "Al" sinyali veren hisseleri göster
    except Exception as e:
        print(f"{hisse} için veri alınamadı: {e}")

# Sinyal veren hisseleri excele kaydetme
df_signals.to_excel("sinyal_veren_hisseler.xlsx", index=False)
print("\nSinyal veren hisseler 'sinyal_veren_hisseler.xlsx' dosyasına kaydedildi.")

  Cloning https://github.com/rongardF/tvdatafeed to /tmp/pip-req-build-8in3suqy
  Running command git clone --filter=blob:none --quiet https://github.com/rongardF/tvdatafeed /tmp/pip-req-build-8in3suqy
  Resolved https://github.com/rongardF/tvdatafeed to commit e6f6aaa7de439ac6e454d9b26d2760ded8dc4923
  Preparing metadata (setup.py) ... done



Lütfen zaman dilimini (interval) seçin:
1: Günlük
2: Haftalık
3: Aylık
Seçiminiz (1/2/3): 1
['BOBET', 0.25, True, True, True, True]
['GLDTR', 2.68, True, True, True, True]
['ISMEN', 0.47, True, True, True, True]
['KOZAA', 0.73, True, True, True, True]
['SNKRN', 0.59, True, True, True, True]
['VKFYO', 0.21, True, True, True, True]

Sinyal veren hisseler 'sinyal_veren_hisseler.xlsx' dosyasına kaydedildi.
